# Data Visualization for US Real GDP and Personal Income by State

_Author_: QH  
_First Version Date: 2022-10-30_  
_Latest update on: 2022-11-21_

This project aims to extract real GDP and Personal data from US Bureau of Economic Analysis (BEA) API and have a map presentation of the real GDP and Personal Income over time by states.

This notebook will follow the steps:
* Download Real GDP by States and Industry from BEA API
* Download Real Personal Income by States from BEA API
* Data Processing to extract needed information for both datasets
* Tableau Public Visualization

## Python and Library Versions

Import the packages needed for the analysis and print out the versions.

In [42]:
import requests
import json
import pandas as pd
import numpy as np
import time

# import utility functions
import sys
# setting path
sys.path.append('..')
from py_util import util, preprocessing_util as prep

In [88]:
print('Python version: ' + sys.version)
package_dict = {'Pandas': pd, 'Numpy': np, 'Requests': requests, 'JSON': json}
util.print_version(package_dict)

Python version: 3.9.4 (v3.9.4:1f2e3088f3, Apr  4 2021, 12:32:44) 
[Clang 6.0 (clang-600.0.57)]
Pandas: 1.4.3
Numpy: 1.23.2
Requests: 2.28.0
JSON: 2.0.9


## Get information from BEA API

BEA has an API Data Retrieval guide: https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf that can help to navigate to find the right table and specify the right parameters to grab the data.

The following example requests are very useful:
* `GetParameterValues` Example Request 2: 
    * https://apps.bea.gov/api/data?&UserID=Your-36CharacterKey&method=GetParameterValues&datasetname=Regional&ParameterName=TableName&ResultFormat=XML
    * This request lists all the regional tables that can be retrieved from the API. From which, we are interested in the following tables:
        * `ParamValue Desc="Real GDP by state (NAICS)" Key="SAGDP9N"`
        * `ParamValue Desc="Personal Income Summary: Personal Income, Population, Per Capita Personal Income (Non-Industry)" key="SAINC1"`
        * `ParamValue Desc="Real Personal Income by state (Non-Industry)" Key="SARPI"`
* `GeParameterValuesFiltered` Example Request:
    * http://apps.bea.gov/api/data?&UserID=Your-36CharacterKey&method=GetParameterValuesFiltered&datasetname=Regional&TargetParameter=LineCode&TableName=SAINC1&ResultFormat=XML
    * We can use this to get the potential values of the tables we are interested in.
* `GetData` Example Request 1:
    * https://apps.bea.gov/api/data/?&UserID=Your-36CharacterKey&method=GetData&datasetname=Regional&TableName=CAINC1&LineCode=3&GeoFIPS=DE&Year=2014&ResultFormat=XML
    * This example is helpful to retrieve data. The TableId and LineCode parameters are used to request statistic for “Per Capita personal income (county annual income)”. The GeoFIPS parameter value is “DE” – meaningthe data for all counties in Delaware are requested. A single year’s data is requested – 2014.

In [5]:
# @hidden_cell
USER_ID = "4E170F20-9B82-45BB-B20E-14C5EC5D127D"

### Get all possible values for LineCode, GeoFIPS and Year
This section will we find out all the values of the LineCode (Category), GeoFIPs (State/County) and Years each table we are interested in have. We will define a helper function to achieve that.

In [13]:
def get_param_value(tbname, param):
    parameter = {
        "user_id": USER_ID,
        "method": "GetParameterValuesFiltered",
        "dsName": 'Regional',
        'tblName': tbname,
        'pmName': param,
        "rsltFmt": "JSON",
    }

    base_url = f"https://apps.bea.gov/api/data?&UserID={parameter['user_id']}"
    result = f"&ResultFormat={parameter['rsltFmt']}"
    method = f"&method={parameter['method']}"
    datasetname = f"&datasetname={parameter['dsName']}"
    paramname = f"&TargetParameter={parameter['pmName']}"
    tablename = f"&TableName={parameter['tblName']}"

    url = base_url + method + datasetname + tablename + paramname + result
    print(f"Getting {param} values for {tbname}...")

    r = requests.get(url)
    files = r.json()
    df_out = pd.DataFrame(files['BEAAPI']['Results']['ParamValue'])

    return df_out

In [19]:
# GDP data
# LineCode
SAGDP9N_linecode = get_param_value('SAGDP9N', 'LineCode')
# Save to a csv file
SAGDP9N_linecode.to_csv('data/SAGDP9N_linecode.csv', index=False)
# Year
SAGDP9N_year = get_param_value('SAGDP9N', 'year')
# Save to a csv file
SAGDP9N_year.to_csv('data/SAGDP9N_year.csv', index=False)
# GeoFips
SAGDP9N_GeoFips = get_param_value('SAGDP9N', 'GeoFips')
# Save to a csv file
SAGDP9N_GeoFips.to_csv('data/SAGDP9N_GeoFips.csv', index=False)

Getting LineCode values for SAGDP9N...
Getting year values for SAGDP9N...
Getting GeoFips values for SAGDP9N...


In [20]:
# Personal Income Data
# LineCode
SAINC1_linecode = get_param_value('SAINC1', 'LineCode')
# Save to a csv file
SAINC1_linecode.to_csv('data/SAINC1_linecode.csv', index=False)
# Year
SAINC1_year = get_param_value('SAINC1', 'year')
# Save to a csv file
SAINC1_year.to_csv('data/SAINC1_year.csv', index=False)
# GeoFips
SAINC1_GeoFips = get_param_value('SAINC1', 'GeoFips')
# Save to a csv file
SAINC1_GeoFips.to_csv('data/SAINC1_GeoFips.csv', index=False)

Getting LineCode values for SAINC1...
Getting year values for SAINC1...
Getting GeoFips values for SAINC1...


In [21]:
# Real Personal Income Data
# LineCode
SARPI_linecode = get_param_value('SARPI', 'LineCode')
# Save to a csv file
SARPI_linecode.to_csv('data/SARPI_linecode.csv', index=False)
# Year
SARPI_year = get_param_value('SARPI', 'year')
# Save to a csv file
SARPI_year.to_csv('data/SARPI_year.csv', index=False)
# GeoFips
SARPI_GeoFips = get_param_value('SARPI', 'GeoFips')
# Save to a csv file
SARPI_GeoFips.to_csv('data/SARPI_GeoFips.csv', index=False)

Getting LineCode values for SARPI...
Getting year values for SARPI...
Getting GeoFips values for SARPI...


## Get Data

Since we cannot specify all for LineCode, Year and GeoFips for all regional tables to retrieve information, we will extract by GeoFips. First, we created a helper function to extract data by GeoFips.

In [41]:
def get_data(tbname, geoFIPs):
    parameter = {
        "user_id": USER_ID,
        "method": "GetData",
        "dsName": 'Regional',
        'tblName': tbname,
        "geoFIPs": geoFIPs,
        "LineCode": 'ALL',
        "Year": 'ALL',
        "rsltFmt": "JSON",
    }

    base_url = f"https://apps.bea.gov/api/data?&UserID={parameter['user_id']}"
    result = f"&ResultFormat={parameter['rsltFmt']}"
    method = f"&method={parameter['method']}"
    datasetname = f"&datasetname={parameter['dsName']}"
    tablename = f"&TableName={parameter['tblName']}"
    geoFips = f"&GeoFIPS={parameter['geoFIPs']}"
    linecode = f"&LineCode={parameter['LineCode']}"
    year = f"&Year={parameter['Year']}"

    url = base_url + method + datasetname + tablename + geoFips + linecode + year + result
    print(f"Getting {geoFIPs}...")

    r = requests.get(url)
    files = r.json()
    df_out = pd.DataFrame(files['BEAAPI']['Results']['Data'])
    print(f"{geoFIPs} imported.")
    return df_out

In [24]:
# Specify all the states/districts we want to extract
all_code = ["{:05d}".format(i) for i in range(0, 57000, 1000)]
excl_code = ['03000', '07000', '14000', '43000', '52000']
state_code = [i for i in all_code if i not in excl_code]
print(state_code)
print(len(state_code))

['00000', '01000', '02000', '04000', '05000', '06000', '08000', '09000', '10000', '11000', '12000', '13000', '15000', '16000', '17000', '18000', '19000', '20000', '21000', '22000', '23000', '24000', '25000', '26000', '27000', '28000', '29000', '30000', '31000', '32000', '33000', '34000', '35000', '36000', '37000', '38000', '39000', '40000', '41000', '42000', '44000', '45000', '46000', '47000', '48000', '49000', '50000', '51000', '53000', '54000', '55000', '56000']
52


### Get Real GDP Data
First, we will extract the real GDP data from SAGDP9N table.

In [25]:
SAGDP9N_total = pd.DataFrame()
for i in range(52):
    SAGDP9N_total = pd.concat([SAGDP9N_total, get_data('SAGDP9N', state_code[i])], ignore_index=True)

Getting 00000...
00000 imported.
Getting 01000...
01000 imported.
Getting 02000...
02000 imported.
Getting 04000...
04000 imported.
Getting 05000...
05000 imported.
Getting 06000...
06000 imported.
Getting 08000...
08000 imported.
Getting 09000...
09000 imported.
Getting 10000...
10000 imported.
Getting 11000...
11000 imported.
Getting 12000...
12000 imported.
Getting 13000...
13000 imported.
Getting 15000...
15000 imported.
Getting 16000...
16000 imported.
Getting 17000...
17000 imported.
Getting 18000...
18000 imported.
Getting 19000...
19000 imported.
Getting 20000...
20000 imported.
Getting 21000...
21000 imported.
Getting 22000...
22000 imported.
Getting 23000...
23000 imported.
Getting 24000...
24000 imported.
Getting 25000...
25000 imported.
Getting 26000...
26000 imported.
Getting 27000...
27000 imported.
Getting 28000...
28000 imported.
Getting 29000...
29000 imported.
Getting 30000...
30000 imported.
Getting 31000...
31000 imported.
Getting 32000...
32000 imported.
Getting 33

In [26]:
SAGDP9N_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119600 entries, 0 to 119599
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Code         119600 non-null  object
 1   GeoFips      119600 non-null  object
 2   GeoName      119600 non-null  object
 3   TimePeriod   119600 non-null  object
 4   Description  119600 non-null  object
 5   CL_UNIT      119600 non-null  object
 6   UNIT_MULT    119600 non-null  object
 7   DataValue    119600 non-null  object
 8   NoteRef      59 non-null      object
dtypes: object(9)
memory usage: 8.2+ MB


In [27]:
SAGDP9N_total.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,SAGDP9N-1,00000,United States,1997,All industry total,Millions of chained 2012 dollars,6,"11,529,157.0",NaN
1,SAGDP9N-1,00000,United States,1998,All industry total,Millions of chained 2012 dollars,6,"12,045,824.0",NaN
2,SAGDP9N-1,00000,United States,1999,All industry total,Millions of chained 2012 dollars,6,"12,623,361.0",NaN
3,SAGDP9N-1,00000,United States,2000,All industry total,Millions of chained 2012 dollars,6,"13,138,035.0",NaN
4,SAGDP9N-1,00000,United States,2001,All industry total,Millions of chained 2012 dollars,6,"13,263,417.0",NaN


As can be seen from the data above, some pre-processing is needed. For example, `DataValue` columnd is a string column instead of numeric. `Description` column may contain all the industry levels and we need to filter to get the top two-level NAIC (North American Industry Code) for the summary.

In [28]:
# First, change DataValue to be numeric/floating
realgdp = SAGDP9N_total['DataValue'].str.replace(',','').str.replace('\(L\)', '', regex=True)
SAGDP9N_total['RealGDP'] = realgdp.replace('', np.NaN).astype("float")
SAGDP9N_total['Description'] = SAGDP9N_total['Description'].str.strip()
SAGDP9N_total.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,NoteRef,RealGDP
0,SAGDP9N-1,00000,United States,1997,All industry total,Millions of chained 2012 dollars,6,"11,529,157.0",NaN,11529157.0
1,SAGDP9N-1,00000,United States,1998,All industry total,Millions of chained 2012 dollars,6,"12,045,824.0",NaN,12045824.0
2,SAGDP9N-1,00000,United States,1999,All industry total,Millions of chained 2012 dollars,6,"12,623,361.0",NaN,12623361.0
3,SAGDP9N-1,00000,United States,2000,All industry total,Millions of chained 2012 dollars,6,"13,138,035.0",NaN,13138035.0
4,SAGDP9N-1,00000,United States,2001,All industry total,Millions of chained 2012 dollars,6,"13,263,417.0",NaN,13263417.0


In [29]:
# Split the data to be state level and US overall level
SAGDP9N_US = SAGDP9N_total[SAGDP9N_total['GeoFips'] == '00000']
SAGDP9N_state = SAGDP9N_total[SAGDP9N_total['GeoFips'] != '00000']

In [31]:
# the sum of the total is not equal to the overal US number -> It is because of using the chained US dollar for Real GDP calculation
us_level_overall =SAGDP9N_US[SAGDP9N_US['Description'].str.strip() == 'All industry total'][['TimePeriod', 'Description','RealGDP']]
SAGDP9N_state_grouped = SAGDP9N_state.groupby(['TimePeriod', 'Description'], as_index=False)['RealGDP'].sum()
state_level_overall = SAGDP9N_state_grouped[SAGDP9N_state_grouped['Description'] == 'All industry total'].rename(columns={'RealGDP': 'RealGDP_state_agg'})

check_us_state = us_level_overall.merge(state_level_overall, on=['TimePeriod', 'Description'])
check_us_state

,TimePeriod,Description,RealGDP,RealGDP_state_agg
0,1997,All industry total,11529157.0,11651476.3
1,1998,All industry total,12045824.0,12153637.8
2,1999,All industry total,12623361.0,12722195.5
3,2000,All industry total,13138035.0,13238794.7
4,2001,All industry total,13263417.0,13347174.9
5,2002,All industry total,13488357.0,13585532.9
6,2003,All industry total,13865519.0,13936974.7
7,2004,All industry total,14399696.0,14463721.6
8,2005,All industry total,14901269.0,14963146.0
9,2006,All industry total,15315943.0,15389483.8


We will use the second level NAICs number for the visualization so that we can take a deeper look at the industry composition by state. However, since we noticed above that simply adding the granular segments will not result in the same value as the overall value, we will do the following to perform an adjustment in order to keep the overall real gdp by state same as the `All industry total` category while industry contribution percentage also stay the same by using the granular segments:
* Step 1: For each state and year, keep the top two-level NAIC category and calculate the sum of all the two-level NAIC real GDP value
* Step 2: For each state and year, calculate the percentage of each NAIC category real GDP percentage using the aggregated value
* Step 3: For each state and year, adjust the real GDP for each NAIC using Real GDP from `All industry total` times the percentage calculated in step 2.


In [32]:
# Second level NAIC code from the 
Line_Desc_List = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 
'Other services (except government and government enterprises)', 'Accommodation and food services', 'Arts, entertainment, and recreation',
'Health care and social assistance', 'Educational services', 'Administrative and support and waste management and remediation services',
'Management of companies and enterprises', 'Professional, scientific, and technical services',
'Real estate and rental and leasing','Finance and insurance', 'Information', 'Transportation and warehousing','Retail trade', 
'Wholesale trade', 'Manufacturing', 'Construction', 'Utilities', 'Government and government enterprises']

In [33]:
real_gdp_state_selected = SAGDP9N_state[SAGDP9N_state['Description'].isin(Line_Desc_List)]
# Sum all the industry GDP
real_gdp_state_total = real_gdp_state_selected.groupby(['GeoFips', 'GeoName', 'TimePeriod'], as_index=False)['RealGDP'].sum().rename(columns={'RealGDP':'OrigTotalGDP'})
# Reported all industry GDP
reported_total = SAGDP9N_state[SAGDP9N_state['Description'] == 'All industry total'][['GeoFips', 'GeoName', 'TimePeriod', 'RealGDP']].rename(columns={'RealGDP':'ReportedTotalGDP'})
# Merge these two
gdp_state_total = real_gdp_state_total.merge(reported_total, on=['GeoFips', 'GeoName', 'TimePeriod'], how='inner')
# Merge the total with the selected gdp data
real_gdp_state_selected = real_gdp_state_selected.merge(gdp_state_total, on=['GeoFips', 'GeoName', 'TimePeriod'], how='inner')
# Calculate the percentage for each NAICs category
real_gdp_state_selected['Ind_perc'] = real_gdp_state_selected['RealGDP'] / real_gdp_state_selected['OrigTotalGDP']
# Adjust the GDP based on the industry percentage and reported total
real_gdp_state_selected['RealGDP_Adj'] = real_gdp_state_selected['Ind_perc'] * real_gdp_state_selected['ReportedTotalGDP']
real_gdp_state_selected.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,NoteRef,RealGDP,OrigTotalGDP,ReportedTotalGDP,Ind_perc,RealGDP_Adj
0,SAGDP9N-3,01000,Alabama,1997,"Agriculture, forestry, fishing and hunting",Millions of chained 2012 dollars,6,"2,176.8",NaN,2176.8,146516.5,144501.2,0.014857,2146.858628
1,SAGDP9N-6,01000,Alabama,1997,"Mining, quarrying, and oil and gas extraction",Millions of chained 2012 dollars,6,"3,492.8",NaN,3492.8,146516.5,144501.2,0.023839,3444.757357
2,SAGDP9N-10,01000,Alabama,1997,Utilities,Millions of chained 2012 dollars,6,"4,150.9",NaN,4150.9,146516.5,144501.2,0.028331,4093.805347
3,SAGDP9N-11,01000,Alabama,1997,Construction,Millions of chained 2012 dollars,6,"9,379.4",NaN,9379.4,146516.5,144501.2,0.064016,9250.388559
4,SAGDP9N-12,01000,Alabama,1997,Manufacturing,Millions of chained 2012 dollars,6,"21,824.0",NaN,21824.0,146516.5,144501.2,0.148953,21523.816013


In [34]:
real_gdp_state_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25500 entries, 0 to 25499
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Code              25500 non-null  object 
 1   GeoFips           25500 non-null  object 
 2   GeoName           25500 non-null  object 
 3   TimePeriod        25500 non-null  object 
 4   Description       25500 non-null  object 
 5   CL_UNIT           25500 non-null  object 
 6   UNIT_MULT         25500 non-null  object 
 7   DataValue         25500 non-null  object 
 8   NoteRef           0 non-null      object 
 9   RealGDP           25500 non-null  float64
 10  OrigTotalGDP      25500 non-null  float64
 11  ReportedTotalGDP  25500 non-null  float64
 12  Ind_perc          25500 non-null  float64
 13  RealGDP_Adj       25500 non-null  float64
dtypes: float64(5), object(9)
memory usage: 2.9+ MB


In [45]:
# Save the dataset
real_gdp_state_selected.to_csv('data/us_state_ind_gdp_ts.csv', index=False)

### Get Personal Income (Nominal and Real)
Then we will get the Personal Income data both nominal and real by state.

In [46]:
# Get the nominal personal income
# Wait for 1 mins to avoid the quota cap.
time.sleep(61)
SAINC1_total = pd.DataFrame()
for i in range(52):
    SAINC1_total = pd.concat([SAINC1_total, get_data('SAINC1', state_code[i])], ignore_index=True)

Getting 00000...
00000 imported.
Getting 01000...
01000 imported.
Getting 02000...
02000 imported.
Getting 04000...
04000 imported.
Getting 05000...
05000 imported.
Getting 06000...
06000 imported.
Getting 08000...
08000 imported.
Getting 09000...
09000 imported.
Getting 10000...
10000 imported.
Getting 11000...
11000 imported.
Getting 12000...
12000 imported.
Getting 13000...
13000 imported.
Getting 15000...
15000 imported.
Getting 16000...
16000 imported.
Getting 17000...
17000 imported.
Getting 18000...
18000 imported.
Getting 19000...
19000 imported.
Getting 20000...
20000 imported.
Getting 21000...
21000 imported.
Getting 22000...
22000 imported.
Getting 23000...
23000 imported.
Getting 24000...
24000 imported.
Getting 25000...
25000 imported.
Getting 26000...
26000 imported.
Getting 27000...
27000 imported.
Getting 28000...
28000 imported.
Getting 29000...
29000 imported.
Getting 30000...
30000 imported.
Getting 31000...
31000 imported.
Getting 32000...
32000 imported.
Getting 33

In [71]:
# Wait for 1 mins to avoid the quota cap.
time.sleep(61)
# Get the real personal income
SARPI_total = pd.DataFrame()
for i in range(52):
    SARPI_total = pd.concat([SARPI_total, get_data('SARPI', state_code[i])], ignore_index=True)

Getting 00000...
00000 imported.
Getting 01000...
01000 imported.
Getting 02000...
02000 imported.
Getting 04000...
04000 imported.
Getting 05000...
05000 imported.
Getting 06000...
06000 imported.
Getting 08000...
08000 imported.
Getting 09000...
09000 imported.
Getting 10000...
10000 imported.
Getting 11000...
11000 imported.
Getting 12000...
12000 imported.
Getting 13000...
13000 imported.
Getting 15000...
15000 imported.
Getting 16000...
16000 imported.
Getting 17000...
17000 imported.
Getting 18000...
18000 imported.
Getting 19000...
19000 imported.
Getting 20000...
20000 imported.
Getting 21000...
21000 imported.
Getting 22000...
22000 imported.
Getting 23000...
23000 imported.
Getting 24000...
24000 imported.
Getting 25000...
25000 imported.
Getting 26000...
26000 imported.
Getting 27000...
27000 imported.
Getting 28000...
28000 imported.
Getting 29000...
29000 imported.
Getting 30000...
30000 imported.
Getting 31000...
31000 imported.
Getting 32000...
32000 imported.
Getting 33

In [72]:
data_value = SARPI_total['DataValue'].str.replace(',','').str.replace('\(L\)', '', regex=True)
SARPI_total['PI'] = data_value.replace('', np.NaN).astype("float")
SARPI_total['Description'] = SARPI_total['Description'].str.replace('[0-9]{1}\/', '', regex=True).str.strip()
SARPI_US = SARPI_total[SARPI_total['GeoFips'] == '00000']
SARPI_state = SARPI_total[SARPI_total['GeoFips'] != '00000']
SARPI_state.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,PI
52,SARPI-1,01000,Alabama,2008,Real personal income (millions of constant (20...,Millions of constant 2012 dollars,6,"187,336.6",187336.6
53,SARPI-1,01000,Alabama,2009,Real personal income (millions of constant (20...,Millions of constant 2012 dollars,6,"187,152.4",187152.4
54,SARPI-1,01000,Alabama,2010,Real personal income (millions of constant (20...,Millions of constant 2012 dollars,6,"187,279.9",187279.9
55,SARPI-1,01000,Alabama,2011,Real personal income (millions of constant (20...,Millions of constant 2012 dollars,6,"189,172.7",189172.7
56,SARPI-1,01000,Alabama,2012,Real personal income (millions of constant (20...,Millions of constant 2012 dollars,6,"190,450.8",190450.8


In [73]:
SARPI_state['Description'].value_counts()

Real personal income (millions of constant (2012) dollars)    663
Real PCE (millions of constant (2012) dollars)                663
Real per capita personal income (constant (2012) dollars)     663
Real per capita PCE (constant (2012) dollars)                 663
Name: Description, dtype: int64

In [74]:
SARPI_state_selected = SARPI_state[SARPI_state['Description'].isin(['Real per capita PCE (constant (2012) dollars)', 'Real per capita personal income (constant (2012) dollars)'])]
SARPI_state_selected['Description'].value_counts()


Real per capita personal income (constant (2012) dollars)    663
Real per capita PCE (constant (2012) dollars)                663
Name: Description, dtype: int64

In [84]:
SARPI_state_selected.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,PI
78,SARPI-2,01000,Alabama,2008,Real per capita personal income (constant (201...,Constant 2012 dollars,0,"39,705",39705.0
79,SARPI-2,01000,Alabama,2009,Real per capita personal income (constant (201...,Constant 2012 dollars,0,"39,335",39335.0
80,SARPI-2,01000,Alabama,2010,Real per capita personal income (constant (201...,Constant 2012 dollars,0,"39,135",39135.0
81,SARPI-2,01000,Alabama,2011,Real per capita personal income (constant (201...,Constant 2012 dollars,0,"39,414",39414.0
82,SARPI-2,01000,Alabama,2012,Real per capita personal income (constant (201...,Constant 2012 dollars,0,"39,540",39540.0


In [75]:
data_value = SAINC1_total['DataValue'].str.replace(',','').str.replace('\(NA\)', '', regex=True)
SAINC1_total['PI'] = data_value.replace('', np.NaN).astype("float")
SAINC1_total['Description'] = SAINC1_total['Description'].str.replace('[0-9]{1}\/', '', regex=True).str.strip()
SAINC1_US = SAINC1_total[SAINC1_total['GeoFips'] == '00000']
SAINC1_state = SAINC1_total[SAINC1_total['GeoFips'] != '00000']
SAINC1_state.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,NoteRef,PI
279,SAINC1-1,01000,Alabama,1929,Personal income (millions of dollars),Millions of dollars,6,843.2,NaN,843.2
280,SAINC1-1,01000,Alabama,1930,Personal income (millions of dollars),Millions of dollars,6,697.5,NaN,697.5
281,SAINC1-1,01000,Alabama,1931,Personal income (millions of dollars),Millions of dollars,6,583.7,NaN,583.7
282,SAINC1-1,01000,Alabama,1932,Personal income (millions of dollars),Millions of dollars,6,421.9,NaN,421.9
283,SAINC1-1,01000,Alabama,1933,Personal income (millions of dollars),Millions of dollars,6,435.6,NaN,435.6


In [76]:
SAINC1_state['Description'].value_counts()

Personal income (millions of dollars)    4743
Per capita personal income (dollars)     4743
Population (persons)                     4743
Name: Description, dtype: int64

In [77]:
SAINC1_state_selected = SAINC1_state[SAINC1_state['Description'].isin(['Per capita personal income (dollars)', 'Population (persons)'])]
SAINC1_state_selected['Description'].value_counts()

Per capita personal income (dollars)    4743
Population (persons)                    4743
Name: Description, dtype: int64

In [79]:
us_pi_state = pd.concat([SAINC1_state_selected,SARPI_state_selected], ignore_index=True)
us_pi_state = us_pi_state.sort_values(by=['TimePeriod', 'GeoFips', 'GeoName', 'Description'])
us_pi_state.head()

,Code,GeoFips,GeoName,TimePeriod,Description,CL_UNIT,UNIT_MULT,DataValue,NoteRef,PI
0,SAINC1-3,01000,Alabama,1929,Per capita personal income (dollars),Dollars,0,319,NaN,319.0
93,SAINC1-2,01000,Alabama,1929,Population (persons),Number of persons,0,"2,644,000",NaN,2644000.0
186,SAINC1-3,02000,Alaska *,1929,Per capita personal income (dollars),Dollars,0,(NA),(NA),NaN
279,SAINC1-2,02000,Alaska *,1929,Population (persons),Number of persons,0,(NA),(NA),NaN
372,SAINC1-3,04000,Arizona,1929,Per capita personal income (dollars),Dollars,0,598,NaN,598.0
...,...,...,...,...,...,...,...,...,...,...
8835,SAINC1-2,53000,Washington,1929,Population (persons),Number of persons,0,"1,555,000",NaN,1555000.0
8928,SAINC1-3,54000,West Virginia,1929,Per capita personal income (dollars),Dollars,0,459,NaN,459.0
9021,SAINC1-2,54000,West Virginia,1929,Population (persons),Number of persons,0,"1,717,000",NaN,1717000.0
9114,SAINC1-3,55000,Wisconsin,1929,Per capita personal income (dollars),Dollars,0,670,NaN,670.0


In [94]:
us_pi_state_pivot = us_pi_state.pivot(index=['TimePeriod', 'GeoFips', 'GeoName'], columns='Description', values='PI').reset_index()
us_pi_state_pivot

Description,TimePeriod,GeoFips,GeoName,Per capita personal income (dollars),Population (persons),Real per capita PCE (constant (2012) dollars),Real per capita personal income (constant (2012) dollars)
0,1929,01000,Alabama,319.0,2644000.0,NaN,NaN
1,1929,02000,Alaska *,NaN,NaN,NaN,NaN
2,1929,04000,Arizona,598.0,430000.0,NaN,NaN
3,1929,05000,Arkansas,303.0,1852000.0,NaN,NaN
4,1929,06000,California,994.0,5531000.0,NaN,NaN
...,...,...,...,...,...,...,...
4764,2021,51000,Virginia,66305.0,8642274.0,NaN,NaN
4765,2021,53000,Washington,73775.0,7738692.0,NaN,NaN
4766,2021,54000,West Virginia,48488.0,1782959.0,NaN,NaN
4767,2021,55000,Wisconsin,59626.0,5895908.0,NaN,NaN


In [96]:
us_pi_state.to_csv('data/us_state_pi_ts.csv', index=False)

## Visualization in Tableau
https://public.tableau.com/app/profile/qianyi8267/viz/USRealGDPbyStateandIndustry/USRealGDPandPersonalIncomebyState